In [1]:
import os
import random
import shutil
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## TEST SET GENERATE

In [3]:
def cnt_dir(dir):

    lst = os.listdir(dir)
    cnt = len(lst)

    return cnt

In [12]:
import os

file_name = "/content/gdrive/MyDrive/cv/test.zip"
output_dir = "/content/gdrive/MyDrive/cv"
os.system("unzip "+file_name+" -d "+output_dir)

256

In [16]:
train_neg_path = '/content/gdrive/MyDrive/cv/test/0'
train_pos_path = '/content/gdrive/MyDrive/cv/test/1'

print('test 0 폴더 파일 개수: {}\ntest 1 폴더 파일 개수: {}'.format(cnt_dir(train_neg_path), cnt_dir(train_pos_path)))

test 0 폴더 파일 개수: 1964
test 1 폴더 파일 개수: 3796


## INFERENCE

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(64, 128, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(128, 256, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(256, 512, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(512*7*88, 256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # 마지막 레이어에서 Softmax 없이 출력
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [15]:
# 클래스에 맞게 수정해주세요.
classes = ['하락', '상승']  # 예를 들어, 0과 1로 분류된 클래스 이름

# 데이터 폴더 경로
test_data_path = '/content/gdrive/MyDrive/cv/test'  # 테스트 데이터 폴더 경로

# 이미지를 불러오고 전처리하는 함수 정의
data_transforms = transforms.Compose([
    transforms.Resize((180, 96)),
    transforms.ToTensor(),
])

# 테스트 데이터셋 설정
test_dataset = datasets.ImageFolder(root=test_data_path, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



# 모델 불러오기
model = CNN()  # YourModel()에는 사용하려는 모델이 들어갑니다.
model.load_state_dict(torch.load('/content/gdrive/MyDrive/model1.pth'))
model.eval()

# GPU를 사용할 경우
if torch.cuda.is_available():
    model.to('cuda')

In [19]:
predicted_labels = []
true_labels = []
image_paths = []  # 이미지 파일 경로를 저장할 리스트

with torch.no_grad():
    for images, labels in test_loader:
        if torch.cuda.is_available():
            images = images.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.numpy())

        # 이미지 파일 경로 저장
        image_paths.extend(test_dataset.imgs)  # test_dataset.imgs에 이미지 파일 경로와 레이블이 저장됩니다

# 예측된 클래스 출력 및 정확도 계산
correct_count = 0
total_count = len(true_labels)

for i, (predicted_label, true_label, (image_path, _)) in enumerate(zip(predicted_labels, true_labels, image_paths)):
    image_name = os.path.basename(image_path)  # 파일 경로로부터 파일 이름을 가져옵니다
    correct = 'Correct' if predicted_label == true_label else 'Incorrect'
    print(f"Image {image_name} predicted as: {classes[predicted_label]}, Actual: {classes[true_label]}, Result: {correct}")
    if predicted_label == true_label:
        correct_count += 1

accuracy = correct_count / total_count * 100
print(f"\nTotal Accuracy: {accuracy:.2f}%")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Image GIS_2017-03-27_CV_60.png predicted as: 상승, Actual: 하락, Result: Incorrect
Image GIS_2017-03-27_C_60.png predicted as: 상승, Actual: 하락, Result: Incorrect
Image GIS_2017-06-21_CMA_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GIS_2017-06-21_CVMA_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GIS_2017-06-21_CV_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GIS_2017-06-21_C_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GLW_2017-03-27_CMA_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GLW_2017-03-27_CVMA_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GLW_2017-03-27_CV_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GLW_2017-03-27_C_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GPC_2016-10-03_CMA_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GPC_2016-10-03_CVMA_60.png predicted as: 하락, Actual: 하락, Result: Correct
Image GPC_2016-10-03_CV_60.png